In [ ]:
from pathlib import Path
from itertools import cycle

import geopandas as gpd
from shapely.geometry import box
from rasterio.plot import show
import rasterio
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from shapely.geometry import box
import folium

from src.notebook_helper import calculate_zoom_level, contrast_stretch
from src.util import tif_paths, geojson_paths

# Inspect All Grids
Plot all the grids on a world map

In [ ]:
# Update the path to your folder of grid files
GRIDS_DIR = Path("/Users/kyledorman/data/plant_download/wgs84_all_geojson")
GRID_PATHS = geojson_paths(GRIDS_DIR)

In [ ]:
# Load grids
grids = [gpd.read_file(p) for p in GRID_PATHS]

# Combine into a single GeoDataFrame
global_gdf = gpd.GeoDataFrame(pd.concat(grids, ignore_index=True))

# Convert to local CRS for centroid calculations
local_gdf = global_gdf.to_crs(global_gdf.estimate_utm_crs())

In [ ]:
# Step 1: Sort by centroid position
global_gdf["centroid_x"] = local_gdf.geometry.centroid.x
global_gdf["centroid_y"] = local_gdf.geometry.centroid.y
global_gdf = global_gdf.sort_values(by=["centroid_y", "centroid_x"], ascending=[True, True]).reset_index(drop=True)

# Step 2: Assign colors cyclically
# Generate a large color palette
colors = plt.get_cmap("tab20", 20)  # 20 distinct colors (use larger if needed)
color_palette = [colors(i) for i in range(colors.N)]
hex_colors = [matplotlib.colors.rgb2hex(c[:3]) for c in color_palette]  # Convert to HEX
color_cycle = cycle(hex_colors)

# Assign colors
global_gdf["color"] = [next(color_cycle) for _ in range(len(global_gdf))]

In [ ]:
# Compute the bounding box of all polygons
minx, miny, maxx, maxy = global_gdf.total_bounds

# Calculate the center of the bounding box
center_lat = (miny + maxy) / 2
center_lon = (minx + maxx) / 2

# Calculate dynamic zoom level
zoom_level = calculate_zoom_level(global_gdf.total_bounds)

# Create the base map centered on the calculated location
base_map = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_level, width=800, height=600)

# Add each GeoJSON file to the map
# Add polygons to the map
for _, row in global_gdf.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, color=row["color"]: {
            "fillColor": color,
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.5,
        },
    ).add_to(base_map)

# Display the map
base_map

# Inspect UDM extents compared to an AOI

Set the GRID_ID, YEAR, MONTH variables

In [ ]:
GRID_ID = "25059125"
YEAR = "2023"
MONTH = "10"

In [ ]:
# Update the path to the results directory (e.g. the `results_dir` in the config.yaml file)
RESULTS_DIR = Path("/Users/kyledorman/data/plant_download/test4/results")
GRID_RESULTS_DIR = RESULTS_DIR / YEAR / MONTH / GRID_ID

# Load the UDMS for this grid
UDM_PATHS = geojson_paths(GRID_RESULTS_DIR)

# Load the grid geometry
GRID = gpd.read_file(GRIDS_DIR / f"{GRID_ID}.geojson")

In [ ]:
# Load the UDM GeoJSON file
geojson_file = GRID_RESULTS_DIR / "search_geometries.geojson"
gdf = gpd.read_file(geojson_file)

zoom_level = calculate_zoom_level(GRID.total_bounds)
m = folium.Map(location=(GRID.centroid.iloc[0].y, GRID.centroid.iloc[0].x), zoom_start=zoom_level - 2)

# Add AOI to the map in blue
folium.GeoJson(
    GRID,
    name="AOI",
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "blue",
        "weight": 2,
        "fillOpacity": 0.9,
    },
).add_to(m)

# Plot each polygon with a different color
for _, row in gdf.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda feature, color=next(color_cycle): {
            "fillColor": color,
            "color": color,
            "weight": 2,
            "fillOpacity": 0.05,
        },
    ).add_to(m)


# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m

# Inspect downloaded grid images

To inspect the downloaded imagery, first run
```bash
python src/inspect_grid_outputs.py --month MONTH --year YEAR --config-file CONFIG_FILE --grid-id GRID-ID
```

In [ ]:
asset_dir = GRID_RESULTS_DIR / "files_asset_cropped"
udm_dir = GRID_RESULTS_DIR / "files_udm_cropped"
image_paths = tif_paths(asset_dir)
udm_paths = tif_paths(udm_dir)

cols = min(2, len(image_paths))
rows = len(image_paths) // cols + len(image_paths) % cols
image_size = 5

fig, axes = plt.subplots(rows, cols, figsize=(image_size * cols, image_size * rows))
if cols == rows == 1:
    axes.axis("off")
    axes = [axes]
else:
    axes = axes.flatten()

for i, (img_pth, udm_path, ax) in enumerate(zip(image_paths, udm_paths, axes)):
    with rasterio.open(img_pth) as src:
        img = src.read((7, 5, 3), masked=True)

    with rasterio.open(udm_path) as src:
        udm = (src.read(1) == 1).astype(np.uint8)

    img = contrast_stretch(img, p_high=97, p_low=2)
    show(img, ax=ax)

plt.tight_layout()

In [ ]:
# Create counter
with rasterio.open(udm_paths[0]) as src:
    img = (src.read(1) == 1).astype(np.uint8)
    counter = np.zeros_like(img)

for file in udm_paths:
    with rasterio.open(file) as src:
        img = (src.read(1) == 1).astype(np.uint8)
        counter += img

show(counter, cmap="inferno", title="Counts")
_ = show(counter.clip(0, 5), cmap="inferno", title="Clipped Counts")

# Grid Intersection with regions

In [ ]:
# Load your polygons in WGS84 (EPSG:4326)
polygons_gdf = global_gdf

# Ensure CRS is set to EPSG:4326
polygons_gdf = polygons_gdf.set_crs("EPSG:4326")

# Define the approximate boundary between UTM Zone 10N and Zone 11N (longitude -123° to -117°)
# UTM Zone 10N spans approx. -126° to -120°, and UTM Zone 11N spans approx. -120° to -114°
zone_10_boundary = box(-126, -90, -120, 90)  # Covers all latitudes
zone_11_boundary = box(-120, -90, -114, 90)

# Convert these bounding boxes into GeoDataFrames
zone_10_gdf = gpd.GeoDataFrame(geometry=[zone_10_boundary], crs="EPSG:4326")
zone_11_gdf = gpd.GeoDataFrame(geometry=[zone_11_boundary], crs="EPSG:4326")

# Check intersections
intersect_zone_10 = polygons_gdf[polygons_gdf.intersects(zone_10_boundary)]
intersect_zone_11 = polygons_gdf[polygons_gdf.intersects(zone_11_boundary)]

# Find polygons that intersect BOTH zones
intersect_both = intersect_zone_10[intersect_zone_10.index.isin(intersect_zone_11.index)]

print(f"Number of polygons intersecting both UTM Zone 10 and 11: {len(intersect_both)}")

intersect_both.GRID_ID